In [5]:
!pip install gradio huggingface_hub requests

In [14]:
# === Smart Healthcare GenAI Chatbot (Final Error-Free Version) ===
import requests
import gradio as gr
from huggingface_hub import InferenceClient

# === 🔧 Replace these values with your own ===
HUGGINGFACE_API_KEY = " "   # Your Hugging Face token
THINGSPEAK_CHANNEL_ID = " "                     # Your ThingSpeak Channel ID
THINGSPEAK_READ_KEY = " "              # Your ThingSpeak Read API Key

# ✅ Initialize the model (safe for text_generation)
client = InferenceClient(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    token=HUGGINGFACE_API_KEY
)

# === Fetch latest data from ThingSpeak ===
def fetch_latest_thingspeak(channel_id, read_key=""):
    url = f"https://api.thingspeak.com/channels/{channel_id}/feeds/last.json"
    if read_key:
        url += f"?api_key={read_key}"
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    return {
        "temperature": data.get("field1", "N/A"),
        "spo2": data.get("field2", "N/A"),
        "heart_rate": data.get("field3", "N/A"),
        "updated_at": data.get("created_at", "N/A")
    }

# === Generate automatic health summary ===
def generate_auto_message(readings):
    prompt = f"""
You are a healthcare monitoring AI assistant.
Here are the latest patient readings:
Temperature: {readings['temperature']} °C
SpO₂: {readings['spo2']} %
Heart Rate: {readings['heart_rate']} bpm

Summarize the patient's health in 2–3 sentences.
Say if the condition is normal or needs attention.
End with a short positive suggestion.
"""
    response = client.text_generation(prompt, max_new_tokens=150, temperature=0.6)
    return response.strip()

# === Chat function for user interaction ===
def chatbot(user_input, history):
    readings = fetch_latest_thingspeak(THINGSPEAK_CHANNEL_ID, THINGSPEAK_READ_KEY)
    prompt = f"""
You are a friendly healthcare chatbot.
Patient readings:
Temperature: {readings['temperature']} °C
SpO₂: {readings['spo2']} %
Heart Rate: {readings['heart_rate']} bpm

User asks: "{user_input}"

Provide a short, safe, and kind response.
If any reading looks critical (SpO₂ < 90 or Temp > 39 °C),
advise medical attention.
"""
    reply = client.text_generation(prompt, max_new_tokens=200, temperature=0.7)
    history.append((user_input, reply.strip()))
    return history, history

# === Launch Gradio website ===
def build_and_launch():
    readings = fetch_latest_thingspeak(THINGSPEAK_CHANNEL_ID, THINGSPEAK_READ_KEY)
    auto_msg = generate_auto_message(readings)

    with gr.Blocks(title="Smart Healthcare Chatbot") as demo:
        gr.Markdown("# 🧠 Smart Healthcare AI Assistant")
        gr.Markdown("### 📊 Latest Health Summary")
        gr.Markdown(f"**{auto_msg}**")

        chatbot_ui = gr.Chatbot(label="Chat with your Health Assistant")
        user_box = gr.Textbox(placeholder="Ask about your health...", label="Your Message")
        clear_btn = gr.Button("Clear Chat")

        user_box.submit(chatbot, [user_box, chatbot_ui], [chatbot_ui, chatbot_ui])
        clear_btn.click(lambda: None, None, chatbot_ui, queue=False)

    demo.launch(share=True)  # 🔗 Gives you a public website link

# === Run it ===
build_and_launch()

ValueError: Model mistralai/Mixtral-8x7B-Instruct-v0.1 is not supported for task text-generation and provider together. Supported task: conversational.